In [1]:
import os
os.chdir("/home/newdisk/yangkai/contras_sum")
%load_ext autoreload
%autoreload 2

In [25]:
import tqdm
from scl_data import summarize
def get_raw_data(path):
    raw_data = []
    f = open(path,"r")
    lines = f.readlines()
    f.close()
    for line in tqdm.tqdm_notebook(lines):
        line = line.strip()
        label, text = line.split(',',1)
        label = int(label[1])-1
        text = text[1:-1]
        sum = summarize(text)
        raw_data.append((label,text,sum))

    return raw_data

In [26]:
train_path = "dataset/yelp_multi/yelp_review_full_csv/train.csv"
raw_train_data = get_raw_data(train_path)
test_path = "dataset/yelp_multi/yelp_review_full_csv/test.csv"
raw_test_data = get_raw_data(test_path)

/home/nesa320/anaconda2/envs/contras_sum/lib/python3.6/site-packages/ipykernel_launcher.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


KeyboardInterrupt: 

In [48]:
import torch
torch.save({"train":raw_train_data,"test":raw_test_data}, "dataset/yelp_multi/raw_data.pkl")

In [2]:
import torch
raw_dataset = torch.load("dataset/yelp_multi/raw_data.pkl")

In [3]:
raw_train_data = raw_dataset["train"]
raw_test_data = raw_dataset["test"]

In [4]:
raw_train_data[:10]

[(4,
  "dr. goldberg offers everything i look for in a general practitioner.  he's nice and easy to talk to without being patronizing; he's always on time in seeing his patients; he's affiliated with a top-notch hospital (nyu) which my parents have explained to me is very important in case something happens and you need surgery; and you can get referrals to see specialists without having to see him first.  really, what more do you need?  i'm sitting here trying to think of any complaints i have about him, but i'm really drawing a blank.",
  "he's nice and easy to talk to without being patronizing; he's always on time in seeing his patients; he's affiliated with a top-notch hospital (nyu) which my parents have explained to me is very important in case something happens and you need surgery; and you can get referrals to see specialists without having to see him first."),
 (1,
  "Unfortunately, the frustration of being Dr. Goldberg's patient is a repeat of the experience I've had with so 

In [5]:
import torch
class multiLabelDataset(torch.utils.data.Dataset):
    def __init__(self,raw_data):
        self.data = raw_data

    def __getitem__(self,index):
        return self.data[index]

    def __len__(self):
        return len(self.data)
        
    

In [5]:
train_dataset, test_dataset = torch.load("train_data.pkl")

In [6]:
import random
random.shuffle(raw_train_data)
train_dataset = multiLabelDataset(raw_train_data[:6500])
test_dataset = multiLabelDataset(raw_test_data)

In [7]:
from scl_data import summarize
from transformers import RobertaTokenizer
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
def my_collect(batch):
    batch_size = len(batch)
    perm_index = torch.randperm(batch_size)
    y_a = torch.LongTensor([item[0] for item in batch])
    # print(y_a)
    # print(batch)
    y_b = y_a[perm_index]
    x = [item[1] for item in batch]
    # x_perm = [x[perm_index[i]] for i in range(batch_size)]
    # s = [summarize(x[i]) for i in range(batch_size)]
    s = [item[2] for item in batch]
    s_perm = [s[perm_index[i]] for i in range(batch_size)]
    s_mix = [s[i] + "\n" + s_perm[i] for i in range(batch_size)]

    x_ids = tokenizer(x, padding = 'max_length', max_length = 200, truncation = True, return_tensors="pt")["input_ids"]
    s_mix_ids = tokenizer(s, padding = 'max_length', max_length = 200, truncation = True, return_tensors="pt")["input_ids"]

    return x_ids, s_mix_ids, y_a, y_b
    
    

train_loader = torch.utils.data.DataLoader(train_dataset,batch_size = 4,collate_fn = my_collect, num_workers=2, shuffle = True)
test_loader = torch.utils.data.DataLoader(test_dataset,batch_size = 32,collate_fn = my_collect, num_workers=2, shuffle = True)

In [8]:
from scl_model import scl_model_multi
from transformers import RobertaForSequenceClassification, RobertaConfig
import torch
# device = torch.device(args.gpu_ids)
device = torch.device("cuda:0")
config = RobertaConfig.from_pretrained("roberta-base")
config.num_labels = 5
pretrained_model = RobertaForSequenceClassification.from_pretrained("roberta-base",config=config)
model = scl_model_multi(config,device,pretrained_model,False).to(device)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifie

In [9]:
from opt import OpenAIAdam
optimizer = OpenAIAdam(model.parameters(),
                                  lr=1e-5,
                                  schedule='warmup_linear',
                                  warmup=0.002,
                                  t_total=10000,
                                  b1=0.9,
                                  b2=0.999,
                                  e=1e-08,
                                  l2=0.01,
                                  vector_l2=True,
                                  max_grad_norm=1.0)

In [9]:
torch.save([train_dataset,test_dataset],"train_data.pkl")

In [10]:
def evaluate_model(model, test_loader, recoder, step):
    print("Evaluation Start======")
    model.eval()
    # TP, TN, FN, FP = 0, 0, 0, 0
    
    # step = 0
    bar = tqdm.tqdm(total=len(test_loader))
    bar.update(0)
    correct = 0
    total = 0
    with torch.no_grad():
        for batch in test_loader:
            x_ids, s_mix_ids, y_a, y_b = batch
            seq_ids = x_ids.to(device)
            labels = y_a.to(device)
            logits = model.predict(seq_ids)
            # print(logits)

            prediction = torch.argmax(logits, dim = 1)
            correct += (prediction == labels).sum()
            total += prediction.shape[0]

    acc = correct / total
    print("Acc: ",acc)

    recoder.log_test(acc,step)

In [10]:
import tqdm
class Recoder_multi():
    def __init__(self):
        self.ce_loss_x = []
        self.ce_loss_mix = []
        self.scl_loss = []
        self.loss = []
        self.acc = []
        self.step = []
    def log_train(self,ce_loss_x, ce_loss_mix, scl_loss,loss):
        self.ce_loss_x.append(ce_loss_x.item())
        self.ce_loss_mix.append(ce_loss_mix.item())
        self.scl_loss.append(scl_loss.item())
        self.loss.append(loss.item())
    
    def log_test(self,acc,step):
        self.acc.append(acc)
        self.step.append(step)


    def meter(self,step):
        print("===================================")
        print("loss: ",sum(self.loss)/step)
        print("ce_loss_x: ",sum(self.ce_loss_x)/step)
        print("ce_loss_mix: ",sum(self.ce_loss_mix)/step)
        print("scl_loss: ",sum(self.scl_loss)/step)


step = 0
bar = tqdm.tqdm(total=10000)
bar.update(0)
best_acc = 0
recoder = Recoder_multi()

loss_mask = [1,0,0]

count = 0
begin_eval = False
while(step < 10000):
    model.train()
    for batch in train_loader:
        # optimizer.zero_grad()
        ce_loss_x, ce_loss_mix, scl_loss = model(batch)
        loss = loss_mask[0] * ce_loss_x + loss_mask[1] * ce_loss_mix + loss_mask[2] * scl_loss
        loss.backward()
        print(ce_loss_x.item(), ce_loss_mix.item(), scl_loss.item())
        break

        recoder.log_train(ce_loss_x,ce_loss_mix,scl_loss,loss)

        count += 1
        if (count % 1 == 0):
            # optimizer.step()
            step += 1
            optimizer.zero_grad()

            if (step % 200 == 0):
                begin_eval = True

            if (step % 10 == 0):
                bar.update(10)

        # step += 1
        
        
        if begin_eval:
            recoder.meter(step)
            evaluate_model(model,test_loader,recoder,step)
            torch.save(recoder,"log/run_multi_baseline.pkl")
            
            model.train()
            begin_eval = False
    break

  0%|          | 0/10000 [00:00<?, ?it/s]labels: tensor([[2],
        [1],
        [4],
        [3]], device='cuda:0')
logits: tensor([[1.2500, 1.2206, 1.2354, 1.1286, 1.2204, 1.2200, 1.2245, 1.2008],
        [1.2206, 1.2500, 1.2165, 1.1782, 1.2231, 1.2389, 1.2403, 1.2136],
        [1.2354, 1.2165, 1.2500, 1.1258, 1.2297, 1.2123, 1.2208, 1.1957],
        [1.1286, 1.1782, 1.1258, 1.2500, 1.1510, 1.1935, 1.1820, 1.2097],
        [1.2204, 1.2231, 1.2297, 1.1510, 1.2500, 1.2193, 1.2249, 1.2028],
        [1.2200, 1.2389, 1.2123, 1.1935, 1.2193, 1.2500, 1.2402, 1.2265],
        [1.2245, 1.2403, 1.2208, 1.1820, 1.2249, 1.2402, 1.2500, 1.2193],
        [1.2008, 1.2136, 1.1957, 1.2097, 1.2028, 1.2265, 1.2193, 1.2500]],
       device='cuda:0', grad_fn=<DivBackward0>)
mask: tensor([[0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1.],
        [1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1.,

In [11]:
import torch
batch_size = 4
anchor_count = 2
label = torch.LongTensor([2,3,1,1]).unsqueeze(-1)
mask = label == label.T
mask = mask.repeat(2, 2)

logits_mask = torch.scatter(
            torch.ones_like(mask),
            1,
            torch.arange(batch_size * anchor_count).view(-1, 1),
            0
        )

In [12]:
mask

tensor([[ True, False, False, False,  True, False, False, False],
        [False,  True, False, False, False,  True, False, False],
        [False, False,  True,  True, False, False,  True,  True],
        [False, False,  True,  True, False, False,  True,  True],
        [ True, False, False, False,  True, False, False, False],
        [False,  True, False, False, False,  True, False, False],
        [False, False,  True,  True, False, False,  True,  True],
        [False, False,  True,  True, False, False,  True,  True]])

In [13]:
logits_mask

tensor([[False,  True,  True,  True,  True,  True,  True,  True],
        [ True, False,  True,  True,  True,  True,  True,  True],
        [ True,  True, False,  True,  True,  True,  True,  True],
        [ True,  True,  True, False,  True,  True,  True,  True],
        [ True,  True,  True,  True, False,  True,  True,  True],
        [ True,  True,  True,  True,  True, False,  True,  True],
        [ True,  True,  True,  True,  True,  True, False,  True],
        [ True,  True,  True,  True,  True,  True,  True, False]])

In [14]:
mask * logits_mask

tensor([[False, False, False, False,  True, False, False, False],
        [False, False, False, False, False,  True, False, False],
        [False, False, False,  True, False, False,  True,  True],
        [False, False,  True, False, False, False,  True,  True],
        [ True, False, False, False, False, False, False, False],
        [False,  True, False, False, False, False, False, False],
        [False, False,  True,  True, False, False, False,  True],
        [False, False,  True,  True, False, False,  True, False]])

In [16]:
a = torch.rand([10,120])
b = torch.rand([10,120])
torch.matmul(torch.norm(a,dim = 1,keepdim=True), torch.norm(b,dim=1,keepdim = True).T)

tensor([[41.8977, 39.9862, 39.9220, 38.5843, 37.1067, 38.1846, 38.1274, 41.6253,
         41.9710, 36.4523],
        [42.4863, 40.5480, 40.4828, 39.1263, 37.6279, 38.7210, 38.6630, 42.2101,
         42.5606, 36.9643],
        [41.8212, 39.9132, 39.8490, 38.5138, 37.0389, 38.1148, 38.0577, 41.5493,
         41.8943, 36.3856],
        [43.9832, 41.9765, 41.9091, 40.5048, 38.9536, 40.0852, 40.0252, 43.6972,
         44.0601, 38.2666],
        [43.0596, 41.0951, 41.0291, 39.6543, 38.1357, 39.2435, 39.1848, 42.7797,
         43.1349, 37.4631],
        [40.7969, 38.9357, 38.8731, 37.5705, 36.1317, 37.1813, 37.1257, 40.5317,
         40.8683, 35.4945],
        [42.4008, 40.4664, 40.4013, 39.0476, 37.5522, 38.6431, 38.5852, 42.1252,
         42.4750, 36.8900],
        [40.0300, 38.2037, 38.1423, 36.8642, 35.4525, 36.4824, 36.4277, 39.7697,
         40.1000, 34.8273],
        [43.2007, 41.2298, 41.1635, 39.7842, 38.2607, 39.3721, 39.3132, 42.9199,
         43.2763, 37.5859],
        [40.7945, 3

In [7]:
from transformers import RobertaConfig, RobertaForSequenceClassification
import torch
from scl_model import scl_model_multi
device = torch.device(0)
config = RobertaConfig.from_pretrained("roberta-base")
config.num_labels = 5
pretrained_model = RobertaForSequenceClassification.from_pretrained("roberta-base",config=config)
model_dir = "model/with_mlp_without_mix_scl_pretrain.pkl"
model = scl_model_multi(config,device,pretrained_model,with_semi=False)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifie

In [8]:
model.load_state_dict(torch.load(model_dir))

<All keys matched successfully>